# Setup

## Google Only

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))
sys.path.append('/content/drive/My Drive/Code/autocomplete_me')

In [2]:
!ls -l '/content/drive/My Drive/Code/autocomplete_me/src'

total 19
-rw------- 1 root root 2516 Jul 12 14:10 predict_utils.py
drwx------ 2 root root 4096 Jul 12 14:39 __pycache__
-rw------- 1 root root 2673 Jul 12 14:38 reader.py
-rw------- 1 root root 9341 Jul 12 14:24 utils.py


In [3]:
from src import utils, reader, predict_utils

Using TensorFlow backend.


In [4]:
from importlib import reload
reload(utils)
reload(reader)
reload(predict_utils)

<module 'src.predict_utils' from '/content/drive/My Drive/Code/autocomplete_me/src/predict_utils.py'>

# Build Model

## Training

In [5]:
text = reader.read_trump_tweet()
content_type = 'Twitter - Trump'

47526


In [6]:
text[0]

'New documents just released reveal General Flynn was telling the truth, and the FBI knew it! @OANN'

In [8]:
text = text[:10000] # crashes at 400K

### Short Text

In [9]:
training_dict, word_idx, idx_word, sequences, num_words = utils.get_data(text, training_len=10, short=True)

There are 23406 unique words.
There are 174910 sequences.


In [10]:
len(training_dict)

4

In [11]:
# embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/Users/jaipancholi/data/glove.6B.100d.txt')
embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/content/drive/My Drive/Code/autocomplete_me/data/glove.6B.100d.txt')
embedding_matrix

Glove Vectors loading with dimension 100
There were 15016 words without pre-trained embeddings.


/content/drive/My Drive/Code/autocomplete_me/src/utils.py:180: RuntimeWarning: invalid value encountered in true_divide
  embedding_matrix = embedding_matrix / np.linalg.norm(embedding_matrix, axis=1).reshape((-1, 1))


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.06223089,  0.03835242,  0.08488411, ..., -0.04284497,
         0.08662399, -0.00527513],
       [-0.01938821,  0.01990321,  0.10770387, ..., -0.14973777,
         0.08155941,  0.0148697 ],
       ...,
       [ 0.07559662,  0.09019867, -0.1500041 , ..., -0.0066243 ,
         0.01093789,  0.03745238],
       [ 0.10883924, -0.06250692,  0.10208854, ...,  0.07560617,
        -0.03567615,  0.05948319],
       [ 0.03400484, -0.2261346 , -0.040496  , ...,  0.06427987,
        -0.01206196,  0.06095745]])

#### Unidirectional, 1 layer, Trainable

In [12]:
model = utils.make_word_level_model(
    num_words,
    embedding_matrix,
    lstm_cells=64,
    trainable=True,
    lstm_layers=1,
    bi_direc=False
)

In [14]:
history = utils.train_model(
    training_dict,
    f'{content_type}_uni-1_layer-trainable-20_seq',
    # model=model,
    use_pretrained_model=True,
    epochs=250
)

122436/122436 [==============================] - 15s 124us/step - loss: 2.6609 - accuracy: 0.4122 - val_loss: 12.9202 - val_accuracy: 0.1689
Epoch 148/250
 77824/122436 [==================>...........] - ETA: 4s - loss: 2.6281 - accuracy: 0.4171

UnboundLocalError: ignored

## Compare Model

<tr>
    <th>Name of Model</th>
    <th>LTSM Layers</th>
    <th>Bidirectional</th>
    <th>Trainable Embeddings</th>
    <th>Sequence Length</th>
    <th>LSTM Dropout</th>
    <th>LSTM Recurrent Dropout</th>
    <th>Val Loss</th>
    <th>Val Accuracy</th>
</tr>
<!-- <tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>50</td>
    <td>0.1</td>
    <td>0.1</td>
    <td>6.5235</td>
    <td>0.1715</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>50</td>
    <td>0.2</td>
    <td>0.2</td>
    <td>6.4229</td>
    <td>0.1706</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>50</td>
    <td>0.5</td>
    <td>0.5</td>
    <td>6.2247</td>
    <td>0.1638</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>20</td>
    <td>0.1</td>
    <td>0.1</td>
    <td>6.4449</td>
    <td>0.1725</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>20</td>
    <td>0.2</td>
    <td>0.2</td>
    <td>6.2899</td>
    <td>0.1706</td>
</tr>
<tr>
    <td></td>
    <td>1</td>
    <td>False</td>
    <td>True</td>
    <td>20</td>
    <td>0.5</td>
    <td>0.5</td>
    <td>6.1667</td>
    <td>0.1719</td>
</tr> -->

# Sample Output

## Load Model

In [15]:
from keras.models import load_model
# Get Model Weights and Architecture

# MODELS_DIR = os.path.join(os.path.dirname(os.path.abspath('')), 'models')
MODELS_DIR = '/content/drive/My Drive/Code/autocomplete_me/models'
model_filepath = os.path.join(MODELS_DIR, f'{content_type}_uni-1_layer-trainable-20_seq.h5')

model = load_model(model_filepath)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [17]:
# Get Text Data
TRAINING_LENGTH = 50
training_dict, word_idx, idx_word, sequences, num_words = utils.get_data(text, training_len=TRAINING_LENGTH)

There are 23406 unique words.
There are 22 sequences.


In [30]:
original_sequence, gen_list, a = predict_utils.generate_output(
    model,
    sequences,
    idx_word,
    seed_length=TRAINING_LENGTH,
    new_words=50,
    diversity=1,
    n_gen=1
)

In [31]:
' '.join(word for word in original_sequence)

'us to get out of these ridiculous Endless Wars , many of them tribal , and bring our soldiers home . WE WILL FIGHT WHERE IT IS TO OUR BENEFIT , AND ONLY FIGHT TO WIN . Turkey , Europe , Syria , Iran , Iraq , Russia and the'

In [32]:
' '.join(word for word in gen_list[0])

'< --- > . of This heard •Seeks , and no impacted ,'

In [25]:
sentence = 'Stocks of major large technology firms are becoming'
predict_utils.generate_custom_sentence(sentence, word_idx, idx_word, model, new_words=50)

[4052, 6, 520, 642, 2983, 6522, 15, 2331]
getting honor are did are looking to keep a really Left job long against him that Infrastructure , on ZERO exit on what to make lets come more long Manchin of the foreign hit work with heights Taylor: .taxpayer was great specifically made we want &amp any light of got
